In [228]:
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
pd.options.display.max_rows = 100
pd.options.display.max_columns = 40
import numpy as np
import os, shutil
from tqdm import tqdm

# Warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# Web Scraping
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from time import sleep
from bs4 import BeautifulSoup
from tqdm import tqdm
options = webdriver.ChromeOptions()

options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-gpu")

options.add_argument("window-size=1920x1080")
options.add_argument("lang=ko_KR")
options.add_argument("user-agent=Chrome/77.0.3865.90")

from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
import urllib, requests, json
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import numpy as np
import os, re

# cf.) functions

In [236]:
# 도로명주소 -> 지번주소
def transform_address(address_ls, filepath, output):
    # output 저장할 파일 생성
    with open(output, "w") as f:
        f.write("지번주소;도로명주소;건물명;폐지여부\n")
    
    # 크롤링
    url = "https://www.juso.go.kr/openIndexPage.do"
    driver = webdriver.Chrome(executable_path = 'C:/Program Files/chromedriver/chromedriver.exe',options = options)

    for address in tqdm(address_ls):
        driver.get(url)
        selected_tag_a=driver.find_element_by_css_selector('input#inputSearchAddr')
        selected_tag_a.click()
        selected_tag_a.send_keys(address) 
        selected_tag_a.find_element_by_xpath('//*[@id="AKCFrm"]/fieldset/div/button').send_keys((Keys.ENTER))
        source=driver.page_source
        soup=BeautifulSoup(source,'lxml') 
        # 지번주소, 도로명주소, 건물명, 폐지 여부 웹스크래핑
        try:
            new_address = soup.select('#rnAddr1')[0].get('value').split('(')[0]
            abo = "존재"
        except IndexError: # 검색 결과가 없는 경우
            print(f"{address}은(는) 폐지된 주소 정보입니다.")
            driver.find_element_by_css_selector('input#ckAblYn').click()
            driver.find_element_by_css_selector('button#searchButton').click()
            new_address = soup.select('#rnAddr1')[0].get('value').split('(')[0]
            abo = "폐지"
        except TimeoutException as ex:
            print(ex.Message)
            webDriver.navigate().refresh()
        old_address = soup.select('#lndnAddr1')[0].get('value')
        bdNm = soup.select('#bdNm1')[0].get('value')
        
        # output txt파일에 저장
        if address == old_address:
            old_address = address
        elif address == new_address:
            new_address = address
        with open(os.path.join(filepath, output), "a") as f:
            f.write(f"{old_address};{new_address};{bdNm};{abo}\n")
    # txt파일 불러오기
    trans_address = pd.read_csv(os.path.join(filepath, output), sep=";",encoding ="cp949")
    return trans_address
            
def get_data(xmlobj, tag):
    try:
        result = xmlobj.findAll(tag)[0].text
    except:
        result = re.findall("\d+", str(xmlobj.findAll(tag)[0]))[0]
    return result

def get_archarea(trans_address, new_address, k_var, v_var, API_KEY, platGbCd='0', startDate="", endDate=""):
    old_address = trans_address.loc[trans_address.도로명주소==new_address,"지번주소"].values[0]
    bjdongNm = re.findall('.*?동|.*?리|.*?가', old_address)[0]
    code = bupjeongdong.loc[bupjeongdong.법정동명==bjdongNm,'법정동코드'].unique()[0]
    sigunguCd = str(code)[:5]
    bjdongCd = str(code)[5:]
    try:
        # 지번 주소에 '-'이 있는 경우
        # ex) 336-1 -> bun = 0336, ji = 0001
        bun_ls = old_address.split("-")
        bun = re.findall("\d+", bun_ls[0])[0].zfill(4)
        ji = re.findall("\d+", bun_ls[1])[0].zfill(4)
    except:
        # 지번 주소에 '-'이 없는 경우
        # ex) 695 -> bun = 0695, ji = 0000
        bun = re.findall("\d+", old_address)[0].zfill(4)
        ji = "0000"
        
    # OpenAPI
    API_KEY = API_KEY
    API_URL = 'http://apis.data.go.kr/1613000/BldRgstService_v2/getBrTitleInfo'
    queryParams = '?' + urlencode({quote_plus('serviceKey') : API_KEY, 
                                   quote_plus('sigunguCd') : sigunguCd, 
                                   quote_plus('bjdongCd') : bjdongCd, 
                                   quote_plus('platGbCd') : platGbCd, 
                                   quote_plus('bun') : bun, 
                                   quote_plus('ji') : ji, 
                                   quote_plus('startDate') : startDate, 
                                   quote_plus('endDate') : endDate, 
                                   quote_plus('numOfRows') : '10',
                                   quote_plus('pageNo') : '1' })
    response = requests.get(API_URL+queryParams).text.encode('utf-8')
    xmlobj = BeautifulSoup(response, 'html.parser')
    
    # key
    res_dic = {k: v for k,v in zip(k_var, v_var)}
    # tag
    tag_list = list(map(lambda x: x.lower(), v_var))
    data = [get_data(xmlobj, tag) for tag in tag_list]
   
    # Dataframe
    for idx, i in  enumerate(res_dic.keys()):
        res_dic[i] = [data[idx]]
    df = pd.DataFrame(res_dic)
    df['도로명주소'] = new_address
    df['지번주소'] = old_address

    return df

def get_building(trans_address, address, k_var, v_var, API_KEY):
    building = pd.DataFrame(columns=k_var)
    for i in tqdm(address):
        try:
            res = get_archarea(trans_address,i,k_var,v_var, API_KEY)
        except:
            res = pd.DataFrame(columns=k_var)
        building = pd.concat([building,res])
    building.reset_index(inplace=True)
    del building['index']
    return building  

# 데이콘 데이터셋 - train.csv, test.csv
- new_address_tr - train 도로명주소
- new_address_tst - test 도로명주소

In [237]:
filepath = '../ProcessedData'
train = pd.read_csv(os.path.join(filepath,'train_processed.csv'))
print(train.단지코드.nunique())
print(train.도로명주소.nunique())

421
421


In [238]:
filepath = '../ProcessedData'
test = pd.read_csv(os.path.join(filepath,'test_processed.csv'))
print(test.단지코드.nunique())
print(test.도로명주소.nunique())

150
150


In [240]:
new_address_tr = list(train.도로명주소.unique())
new_address_tst = list(test.도로명주소.unique())
print(train.도로명주소.nunique()+test.도로명주소.nunique())
print(len(list(train.도로명주소.unique())+list(test.도로명주소.unique())))

571
571


In [241]:
# 단지명 기준으로 train, test 중복 처리
dup_list = train.loc[train.단지명.isin(test.단지명.unique()),'단지명'].unique()
print(*dup_list, sep="\n")
danji_tr = {}
danji_tst = {}
for i in dup_list:
    danji_tr[i] = train.loc[train.단지명==i,'단지코드'].unique()[0]
    danji_tst[i] = test.loc[test.단지명==i,'단지코드'].unique()[0]
danji_tr, danji_tst
print(danji_tr)
print(danji_tst)

휴먼시아
송정주공아파트
{'휴먼시아': 'C1379', '송정주공아파트': 'C1569'}
{'휴먼시아': 'C2423', '송정주공아파트': 'C2144'}


In [242]:
# 도로명주소 기준으로 train, test 중복 처리
dup_list = train.loc[train.도로명주소.isin(test.도로명주소.unique()),'도로명주소'].unique()
print(*dup_list, sep="\n")
danji_tr = {}
danji_tst = {}
for i in dup_list:
    danji_tr[i] = train.loc[train.도로명주소==i,'단지코드'].unique()[0]
    danji_tst[i] = test.loc[test.도로명주소==i,'단지코드'].unique()[0]
danji_tr, danji_tst
print(danji_tr)
print(danji_tst)

경기도 용인시 기흥구 흥덕1로79번길 7
{'경기도 용인시 기흥구 흥덕1로79번길 7': 'C1036'}
{'경기도 용인시 기흥구 흥덕1로79번길 7': 'C2675'}


# 공공임대주택 단지정보 - housing.csv

In [243]:
filepath = "../External"
housing = pd.read_csv(os.path.join(filepath,'external_data.csv'))

In [244]:
print(housing.columns)
housing.columns = ['기관명', '광역시도코드', '지역', '시군구코드', '시군구명', '단지명', '도로명주소', '총세대수',
       '공급유형', '형명', '전용면적', '공용면적', '임대건물구분', '주차수', '기본임대보증금', '기본월임대료',
       '기본전환보증금']
print(housing.columns)

Index(['기관명', '광역시도코드', '광역시도명', '시군구코드', '시군구명', '단지명', '도로명주소', '세대수',
       '공급유형명', '형명', '공급전용면적', '공급공용면적', '주택유형명', '주차수', '기본임대보증금', '기본월임대료',
       '기본전환보증금'],
      dtype='object')
Index(['기관명', '광역시도코드', '지역', '시군구코드', '시군구명', '단지명', '도로명주소', '총세대수', '공급유형',
       '형명', '전용면적', '공용면적', '임대건물구분', '주차수', '기본임대보증금', '기본월임대료', '기본전환보증금'],
      dtype='object')


# 도로명주소 -> 지번주소 변환

In [245]:
filepath = "./"
output = 'trans_address_tr.txt'
trans_address_tr = transform_address(new_address_tr, filepath, output)
trans_address_tr

100%|████████████████████████████████████████████████████████████████████████████████| 421/421 [05:08<00:00,  1.36it/s]


,지번주소,도로명주소,건물명,폐지여부
0,경상북도 포항시 북구 양덕동 336-1,경상북도 포항시 북구 장량중앙로 17,장량휴먼시아,존재
1,경상남도 통영시 미수동 695,경상남도 통영시 미우지해안로 107,미수휴먼시아,존재
2,대전광역시 유성구 상대동 465,대전광역시 유성구 상대로 40,도안휴먼시아4단지,존재
3,경기도 남양주시 호평동 712,경기도 남양주시 늘을3로 8,주공Humansia,존재
4,전라북도 익산시 오산면 장신리 1130,전라북도 익산시 오산면 선화로4길 66,장신휴먼시아1단지,존재
...,...,...,...,...
416,제주특별자치도 서귀포시 서홍동 355-1,제주특별자치도 서귀포시 장수로 115,서귀포서홍 행복주택,존재
417,강원도 춘천시 우두동 1088,강원도 춘천시 우두1길 40,춘천우두엘에이치천년나무3단지,존재
418,부산광역시 남구 용호동 376-9,부산광역시 남구 동명로 183,NaN,존재
419,충청북도 제천시 고암동 1273,충청북도 제천시 용두천로40길 28,NaN,존재


In [246]:
filepath = "./"
output = 'trans_address_tst.txt'
trans_address_tst = transform_address(new_address_tst, filepath, output)
trans_address_tst

100%|████████████████████████████████████████████████████████████████████████████████| 150/150 [01:51<00:00,  1.35it/s]


,지번주소,도로명주소,건물명,폐지여부
0,경기도 광명시 소하동 1391,경기도 광명시 서원로 25,광명역세권휴먼시아,존재
1,경기도 광명시 소하동 1373,경기도 광명시 성채안로 26,광명역세권휴먼시아,존재
2,부산광역시 동래구 안락동 1256,부산광역시 동래구 충렬대로447번길 38,부산안락4 휴먼시아,존재
3,전라북도 익산시 오산면 장신리 1126,전라북도 익산시 오산면 선화로4길 14,장신휴먼시아3단지,존재
4,경기도 성남시 분당구 삼평동 728,경기도 성남시 분당구 동판교로 212,봇들마을6단지아파트,존재
...,...,...,...,...
145,제주특별자치도 서귀포시 성산읍 고성리 1141-1,제주특별자치도 서귀포시 성산읍 고성동서로 33,서귀포성산엘에이치아파트,존재
146,충청북도 충주시 호암동 1066,충청북도 충주시 호암수청2로 56,충주호암1단지,존재
147,강원도 화천군 화천읍 신읍리 1818,강원도 화천군 화천읍 산천어길 196,공공실버주택,존재
148,경상남도 창원시 마산합포구 가포동 58-2,경상남도 창원시 마산합포구 가포본동5길 37,창원가포 엘에이치아파트 1단지,존재


# 법정동 - bupjeongdong.csv

In [249]:
filepath = "./"
bupjeongdong = pd.read_csv(os.path.join(filepath, "bupjeongdong.csv"))
bupjeongdong

,법정동코드,법정동명,폐지여부
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재
...,...,...,...
46287,5013032022,제주특별자치도 서귀포시 표선면 하천리,존재
46288,5013032023,제주특별자치도 서귀포시 표선면 성읍리,존재
46289,5013032024,제주특별자치도 서귀포시 표선면 가시리,존재
46290,5013032025,제주특별자치도 서귀포시 표선면 세화리,존재


# 건축물대장정보 - building.csv

In [251]:
API_KEY = 'aaJUNB0G9RL42RIOAUXocxGfIZRGIP9cxVQ+7+IvE8xiQBA6rhn6AAMqQ81eOudK/8Hb6jfX48cDA2BtbsGt4g=='
k_var = ["관리건축물대장PK","건물명","대지면적","건축면적","연면적","용적률산정연면적","용적률","세대수","가구수"]
v_var = ["mgmBldrgstPk","bldNm","platArea","archArea","totArea","vlRatEstmTotArea","vlRat","hhldCnt","fmlyCnt"]

building_tr = get_building(trans_address_tr, new_address_tr,k_var,v_var,API_KEY=API_KEY)
building_tst = get_building(trans_address_tst, new_address_tst,k_var,v_var,API_KEY=API_KEY)

100%|████████████████████████████████████████████████████████████████████████████████| 150/150 [00:22<00:00,  6.67it/s]


In [259]:
filepath = "../External3"
building_tr.to_csv(os.path.join(filepath,"building_tr.csv"),index=False)
building_tst.to_csv(os.path.join(filepath,"building_tst.csv"),index=False)